# XTTS Finetuning for Maltese

## Requirements
- Python 3.10

Takes around 6min and you need to restart the session at the end

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Clone the repository
%rm -rf Malta-TTS
!git clone https://github.com/Wubpooz/Malta-TTS.git
%cd Malta-TTS/FineTuning/NewLanguage

!pip install --upgrade pip
!pip install -r requirements.txt

# !python -m spacy download en_core_web_sm

Mounted at /content/drive
Cloning into 'Malta-TTS'...
remote: Enumerating objects: 723, done.
remote: Counting objects: 100% (332/332), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 723 (delta 223), reused 203 (delta 99), pack-reused 391 (from 1)
Receiving objects: 100% (723/723), 7.41 MiB | 12.59 MiB/s, done.
Resolving deltas: 100% (417/417), done.
/content/Malta-TTS/FineTuning/NewLanguage
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.4 MB/s  0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━

## Parameters

In [1]:
language_code = "mt"
metadata_train_path = "/content/drive/MyDrive/XTTS_Maltese_Data_20KHz/metadata_train.csv"
metadata_eval_path = "/content/drive/MyDrive/XTTS_Maltese_Data_20KHz/metadata_eval.csv"
output_path = "/content/drive/MyDrive/XTTS_Maltese_Training/output"
extended_vocab_size_param = 100000

%env TOKENIZERS_PARALLELISM=false
%env OMP_NUM_THREADS=1
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

env: TOKENIZERS_PARALLELISM=false
env: OMP_NUM_THREADS=1
env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


## Data Preparation

In [ ]:
# @title Takes 35min initialy but after it's saved to GDrive it's instant
# EXCRUATINGLY SLOW, 35min for saving 4900 audio files => concurrency if available


import os
import shutil
import pandas as pd
from tqdm.auto import tqdm
from datasets import load_dataset
from concurrent.futures import ThreadPoolExecutor

# Set the library path so torchcodec can find ffmpeg ?
os.environ['LD_LIBRARY_PATH'] += ":/usr/lib/x86_64-linux-gnu/"

# def save_single_file(example, output_dir, i):
#     audio_filename = example['audio']['path']
#     audio_bytes = example['audio']['bytes']
#     text = example['normalized_text']
#     speaker_id = example['speaker_id']

#     output_audio_path = os.path.join(output_dir, audio_filename)

#     with open(output_audio_path, 'wb') as f:
#       f.write(audio_bytes) # is it 24kHz ?

#     return {
#         'audio_file': os.path.join("wavs", audio_filename),
#         'text': text,
#         'speaker_name': speaker_id
#     }

# def save_dataset_split_concurent(split, filename):
#     data = []
#     with ThreadPoolExecutor(max_workers=16) as executor:
#         futures = [executor.submit(save_single_file, example, wavs_dir, i) for i, example in enumerate(ds[split])]

#         for future in tqdm(futures, desc=f"Processing {split} split"):
#             data.append(future.result())

#     df = pd.DataFrame(data)
#     df.to_csv(os.path.join(output_dir, filename), sep="|", index=False)
#     print(f"Saved {len(df)} files to {filename}")


def save_dataset_split(split, filename, save_audio=True):
  data = []
  for example in tqdm(ds[split].to_list(), desc=f"Processing {split} split"):
    audio_filename = example['audio']['path']
    audio_bytes = example['audio']['bytes']
    text = example['normalized_text']
    speaker_id = example['speaker_id']

    if(save_audio):
      with open(os.path.join(wavs_dir, audio_filename), 'wb') as f:
        # base audio is 16kHz, but XTTS requires 24KHz
        f.write(audio_bytes)

    # Use LJSpeech format (extended)
    # /!\ audio_file shouldn't have extension, else fails | also they should just be filenames, the loader will add wav/ before and .wav after
    name, ext = os.path.splitext(audio_filename)
    audio_file_without_ext = name

    data.append({
      'audio_file': audio_file_without_ext,
      'text': text,
      'normalized_text': text,
      'speaker_name': speaker_id
    })

  df = pd.DataFrame(data)
  df.to_csv(os.path.join(output_dir, filename), sep="|", index=False)
  print(f"Saved {len(df)} files to {filename}")



output_dir = "/content/drive/MyDrive/XTTS_Maltese_Data"
if os.path.exists(output_dir) and os.path.exists(os.path.join(output_dir, "wavs")) and os.path.exists(os.path.join(output_dir, "metadata_train.csv")):
  print(f"Processed dataset already exists at {output_dir}")
else:
  print("Loading dataset from Hugging Face...")
  ds = load_dataset("Bluefir/MASRI_HEADSET_v2")

  os.makedirs(output_dir, exist_ok=True)
  wavs_dir = os.path.join(output_dir, "wavs")
  os.makedirs(wavs_dir, exist_ok=True)
  if os.path.exists(wavs_dir):
    save_audio = False
  else:
    save_audio = True

  print("Preparing and saving dataset files...")
  save_dataset_split("train", "metadata_train.csv", save_audio)
  save_dataset_split("test", "metadata_eval.csv", save_audio)
  print("Dataset preparation complete (saved to Google Drive too).")

Processed dataset already exists at /content/drive/MyDrive/XTTS_Maltese_Data


In [2]:
# @title Save Metadata and Resample audio (takes 2:30min)
import os
import io
import pandas as pd
from tqdm.auto import tqdm
from datasets import load_dataset, Audio
from concurrent.futures import ThreadPoolExecutor
import librosa
import soundfile as sf

# os.environ['LD_LIBRARY_PATH'] += ":/usr/lib/x86_64-linux-gnu/"

TARGET_SAMPLE_RATE = 22050  # XTTS requirement
NUM_WORKERS = 16

def save_and_resample(example, output_dir, resample=True, save_audio=True):
  audio_filename = example['audio']['path']
  audio_bytes = example['audio']['bytes']
  text = example['normalized_text']
  speaker_id = example['speaker_id']

  save_path = os.path.join(wavs_dir, audio_filename)
  base_name = os.path.splitext(os.path.basename(audio_filename))[0]
  out_path = os.path.join(output_dir, f"{base_name}.wav")

  if save_audio:
    # Read HF bytes safely
    with io.BytesIO(audio_bytes) as f:
      y, sr = sf.read(f)

    # Resample if needed
    if resample and sr != TARGET_SAMPLE_RATE:
      y = librosa.resample(y, orig_sr=sr, target_sr=TARGET_SAMPLE_RATE)
      sr = TARGET_SAMPLE_RATE

    sf.write(out_path, y, sr)

  if(save_audio):
    with open(save_path, 'wb') as f:
      f.write(audio_bytes)

  # if(resample):
  #   y, sample_rate = librosa.load(save_path, sr=None)

  #   if sample_rate != TARGET_SAMPLE_RATE:
  #     y = librosa.resample(y, orig_sr=sample_rate, target_sr=TARGET_SAMPLE_RATE)

  #   sf.write(out_path, y, TARGET_SAMPLE_RATE)


  # Use LJSpeech format (extended)
  # /!\ audio_file shouldn't have extension, else fails | also they should just be filenames, the loader will add wav/ before and .wav after
  return {
    'audio_file': base_name,
    'text': text,
    'normalized_text': text,
    'speaker_name': speaker_id
  }

def process_split(split_name, csv_filename, output_wavs_dir, ds, resample=True, save_audio=True):
  print(f"Processing split: {split_name}")
  results = []

  with ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
    futures = [executor.submit(save_and_resample, ex, output_wavs_dir, resample, save_audio) for ex in ds[split_name]]
    for f in tqdm(futures):
      results.append(f.result())

  # Save metadata
  df = pd.DataFrame(results)
  df.to_csv(os.path.join(output_dir, csv_filename), sep="|", index=False)
  print(f"Saved {len(df)} entries to {csv_filename}")



output_dir = "/content/drive/MyDrive/XTTS_Maltese_Data_20KHz"
wavs_dir = os.path.join(output_dir, "wavs")
os.makedirs(wavs_dir, exist_ok=True)

print("Loading dataset from Hugging Face...")
ds = load_dataset("Bluefir/MASRI_HEADSET_v2")
ds = ds.cast_column("audio", Audio(decode=False))

print(f"Resampling to {TARGET_SAMPLE_RATE} and saving...")
process_split("train", "metadata_train.csv", wavs_dir, ds, True, True)
process_split("test", "metadata_eval.csv", wavs_dir, ds, True, True)

print("Dataset saved!")

Loading dataset from Hugging Face...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/310M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/303M [00:00<?, ?B/s]

...

Resampling to 22050 and saving...
Processing split: train


  0%|          | 0/3983 [00:00<?, ?it/s]

Saved 3983 entries to metadata_train.csv
Processing split: test


  0%|          | 0/996 [00:00<?, ?it/s]

Saved 996 entries to metadata_eval.csv
Dataset saved!


In [ ]:
# @title Dataset repartition
import os
import io
import tempfile
import soundfile as sf
from datasets import load_dataset
from datasets import load_dataset, Audio

# Don't decode audio — just keep metadata
ds = load_dataset("Bluefir/MASRI_HEADSET_v2")
ds = ds.cast_column("audio", Audio(decode=False))

text_lengths = []
audio_durations = []

for split in ["train", "test"]:
    print(f"Processing split: {split}")
    for example in ds[split]:
        # Text length
        text_lengths.append(len(example["normalized_text"]))

        # Save audio bytes to temp file, read duration with soundfile
        audio_bytes = example["audio"]["bytes"]
        with tempfile.NamedTemporaryFile(suffix=".wav") as tmpf:
            tmpf.write(audio_bytes)
            tmpf.flush()
            with sf.SoundFile(tmpf.name) as f:
                duration = len(f) / f.samplerate
                audio_durations.append(duration)

print(f"Text length range: {min(text_lengths)} - {max(text_lengths)} characters")
print(f"Audio duration range: {min(audio_durations):.2f} - {max(audio_durations):.2f} seconds")
print(f"Average text length: {sum(text_lengths)/len(text_lengths):.2f} characters")
print(f"Average audio duration: {sum(audio_durations)/len(audio_durations):.2f} seconds")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/310M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/303M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/155M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3983 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/996 [00:00<?, ? examples/s]

Processing split: train
Processing split: test
Text length range: 1 - 188 characters
Audio duration range: 0.62 - 10.89 seconds
Average text length: 65.99 characters
Average audio duration: 4.82 seconds


## Finetuning

In [7]:
# @title redirection
import sys
_original_stdout = sys.stdout
_original_stderr = sys.stderr
_log_file = None

def output_redirect(redirect=True):
  global _log_file
  if not redirect:
    sys.stdout = _original_stdout
    sys.stderr = _original_stderr
    if _log_file:
      _log_file.close()
      _log_file = None
  else:
    import os
    log_path = "/content/drive/MyDrive/XTTS_Maltese_Training/output/full_training.log"
    # Clear the log file by opening in write mode and closing immediately
    if os.path.exists(log_path):
      with open(log_path, "w"):
        pass
    else:

    _log_file = open(log_path, "a", buffering=1)  # line-buffered

    class Tee(object):
      def __init__(self, *streams):
        self.streams = streams
      def write(self, data):
        for s in self.streams:
          s.write(data)
          s.flush()
      def flush(self):
        for s in self.streams:
          s.flush()

    sys.stdout = Tee(_original_stdout, _log_file)
    sys.stderr = Tee(_original_stderr, _log_file)

In [13]:
# @title Training
import os
output_redirect(True)

os.makedirs(output_path, exist_ok=True)
%cd /content/Malta-TTS/FineTuning/NewLanguage

print(f"Finetuning for {language_code}")
!python new_language_training_cli.py \
    --is_download \
    --is_tokenizer_extension \
    --output_path "{output_path}" \
    --metadatas "{metadata_train_path},{metadata_eval_path},{language_code}" \
    --num_epochs 1 \
    --batch_size 1 \
    --grad_acumm 48 \
    --max_audio_length 255995 \
    --max_text_length 200 \
    --weight_decay 1e-2 \
    --lr 5e-6 \
    --save_step 50_000 \
    --version=main \
    --metadata_path "{metadata_train_path}" \
    --language "{language_code}" \
    --extended_vocab_size {extended_vocab_size_param}

# 35min/epoch on one T4 with batch_size=1, grad_acumm=48, audio_length=255995, max_text=200, weight=1e-2

# Default values are:
# batch-size: 3
# grad_acc: 84
# --multi-gpu

print("Finetuning process completed!")
output_redirect(False)

/content/Malta-TTS/FineTuning/NewLanguage
Finetuning for mt
Step 1: Downloading XTTS base model files.
 > Downloading XTTS v-main files...
 > XTTS model files downloaded successfully!
Step 2: Extending the XTTS tokenizer with the new language.
No new tokens to add by heuristic.
Tokenizer saved to /content/drive/MyDrive/XTTS_Maltese_Training/output/tokenizer.json and copied to vocab.json
Backing up checkpoint: /content/drive/MyDrive/XTTS_Maltese_Training/output/model.pth -> model_backup.pth
Resizing checkpoint embeddings: /content/drive/MyDrive/XTTS_Maltese_Training/output/model.pth
/content/Malta-TTS/FineTuning/NewLanguage/tokenizer_extension.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more detail

## Inference

In [19]:
import os
xtts_checkpoint = os.path.join(output_path, "training", "GPT_XTTS_FT-August-19-2025_02+22PM-2ae7a3e", "checkpoint_3984.pth")

xtts_config = os.path.join(output_path, "config.json")
xtts_vocab = os.path.join(output_path, "vocab.json")

tts_text = "Il-kelma Maltija 'bonġu' tfisser 'good morning'."
speaker_audio_file = "/content/drive/MyDrive/XTTS_Maltese_Data_20KHz/wavs/MSRHS_F_01_P02U004_0058.wav"
lang = "mt"
output_file = "/content/drive/MyDrive/XTTS_Maltese_Data/output_maltese.wav"

# tts_text = "Hi, how are you?"
# speaker_audio_file = "/content/drive/MyDrive/english_speaker.mp3"
# lang="en"
# output_file = "/content/drive/MyDrive/XTTS_Maltese_Data/output_english.wav"

%cd /content/Malta-TTS/FineTuning/NewLanguage
!python inference.py \
    --xtts_checkpoint="{xtts_checkpoint}" \
    --xtts_config="{xtts_config}" \
    --xtts_vocab="{xtts_vocab}" \
    --tts_text="{tts_text}" \
    --speaker_audio_file="{speaker_audio_file}" \
    --lang="{lang}" \
    --output_file="{output_file}" \
    --temperature 0.7 \
    --length_penalty 1.0 \
    --repetition_penalty 10.0 \
    --top_k 50 \
    --top_p 0.8

from IPython.display import Audio
Audio(output_file, rate=24000)

/content/Malta-TTS/FineTuning/NewLanguage
Starting inference...
[2025-08-19 15:04:05,844] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2025-08-19 15:04:07,647] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False
Loading model...
Config loaded.
Initializing model...
Loading checkpoint...
/usr/local/lib/python3.11/dist-packages/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be